In [84]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, RidgeCV, LassoCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from math import sqrt

#Read multiple files
import glob

In [85]:
# Read training data
df_train = pd.read_csv('Balance_training.csv')
df_train.shape

(339399, 65)

In [86]:
# Define categorical columns
categorical_columns = ['Vehicle_Performance',
                       'Vehicle_Passive_Restraint',
                       'Policy_Company','Policy_Billing_Code',
                       'Policy_Method_Of_Payment',
                       'Policy_Reinstatement_Fee_Indicator',
                    'Vehicle_Comprehensive_Coverage_Indicator',
                       'Vehicle_Collision_Coverage_Indicator',
                        'Vehicle_Youthful_Driver_Indicator',
                     'Vehicle_Youthful_Good_Student_Code',
                       'Vehicle_Youthful_Driver_Training_Code',
                     'Vehicle_Safe_Driver_Discount_Indicator',
                     'EEA_Liability_Coverage_Only_Indicator',
                     'EEA_Multi_Auto_Policies_Indicator',
                     'EEA_Packaged_Policy_Indicator',
                     'EEA_Full_Coverage_Indicator',
                       'EEA_Agency_Type',
                     'SYS_Renewed',
                     'SYS_New_Business',
                      'Vehicle_Usage',
                      'Vehicle_Anti_Theft_Device','Vehicle_Make_Description']

In [87]:
# Get dummy columns for training dataset
df_train = pd.get_dummies(df_train, columns=categorical_columns, prefix_sep="_")

#Droping columns after feature selection
df_train.drop(['Driver_Total_Female','Driver_Total_Related_To_Insured_Spouse','Vehicle_Performance_Intermediate'],axis=1, inplace=True)

In [88]:
df_train.shape

(339399, 146)

In [89]:
# Devide dataset into training and testing dataset
X = df_train.drop(['Loss_Amount','Frequency','Severity','Loss_Ratio'],axis=1)
Y = df_train['Loss_Amount']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [90]:
# scale input attributes for train dataset
scaler = preprocessing.StandardScaler()
scaler= scaler.fit(X_train)

# Scale input attributes for Train dataset
X_trainscaled = scaler.transform(X_train)

# Scale input attributes for Test dataset
X_testscaled = scaler.transform(X_test)

In [91]:
# Ridge Regularization with scaling
ncoef= np.linspace(0.1,1000, num=100)

l2_cv = RidgeCV(alphas=ncoef, cv=None, store_cv_values=True)     #  , store_cv_values=True , cv=2  alphas=0.1  cv=2
l2_cv.fit(X_trainscaled,Y_train)
y_pred=l2_cv.predict(X_trainscaled)

In [92]:
rms = sqrt(mean_squared_error(y_pred, Y_train))
print('RMSE for training dataset for L2 model is :: ',rms)

RMSE for training dataset for L2 model is ::  2510.5257427425468


In [93]:
ytest_pred=l2_cv.predict(X_testscaled)
t_rms = sqrt(mean_squared_error(ytest_pred, Y_test))
print('RMSE for testing dataset for L2 model is :: ',t_rms)

RMSE for testing dataset for L2 model is ::  2480.362921513746


In [94]:
# Copy data in tabular format
dfObj = pd.DataFrame(columns=['FileName', 'PredictedLogRatio', 'ActualLogRatio','AbsoluteErrorLogRatio','RMSE_Portfolio'])

In [95]:
# Assigning value to table

## Applying predective model on test data
path = r'.\Trainportfolio' # use your path
all_files = glob.glob(path + "/*.csv")


for filename in all_files:
    port_test = pd.read_csv(filename)
    port_test = pd.get_dummies(port_test, columns=categorical_columns, prefix_sep="_")
    
    port_test.drop(['Driver_Total_Female','Driver_Total_Related_To_Insured_Spouse'],axis=1, inplace=True)

    # Align columns for both test and train to avoid mismatch of columns
    df_train, port_test = df_train.align(port_test, join='left', axis=1, fill_value=0)
    
    X_port = port_test.drop(['Loss_Amount','Frequency','Severity','Loss_Ratio'],axis=1)
    Y_port = port_test['Loss_Amount']

    X_portscaled = scaler.transform(X_port)
    
    port_test['Pred_lossamount']=l2_cv.predict(X_portscaled)
    port_test['Pred_lossratio']=port_test['Pred_lossamount']/port_test['Annual_Premium']
    por_rms = sqrt(mean_squared_error(port_test['Pred_lossamount'], Y_port))
    

    port_actual_loss=port_test['Loss_Amount'].sum()
    port_actual_lossratio=port_test['Loss_Ratio'].mean()
    actual_loss_log = np.log(port_actual_lossratio)
    port_predict_loss_ratio=port_test['Pred_lossratio'].mean()

    predict_loss_log = np.log(port_predict_loss_ratio)
    abs_error = port_predict_loss_ratio-port_actual_lossratio
    
    dfObj = dfObj.append({'FileName': filename[filename.rfind('\\')+1:], 'PredictedLogRatio': predict_loss_log, 'ActualLogRatio': actual_loss_log, 
                          'AbsoluteErrorLogRatio': abs_error, 'RMSE_Portfolio': por_rms}, ignore_index=True)



In [96]:
dfObj.to_csv('Results-L2.csv',index = False)

In [97]:
dfObj.head()

,FileName,PredictedLogRatio,ActualLogRatio,AbsoluteErrorLogRatio,RMSE_Portfolio
0,Test400.csv,0.299419,0.284192,0.020387,1782.975714
1,Test401.csv,0.236895,0.180583,0.069392,1548.752545
2,Test402.csv,0.119735,-0.211575,0.317890,2106.417665
3,Test403.csv,0.185111,0.024458,0.178592,1796.853568
4,Test404.csv,0.238392,-0.322801,0.545088,1731.658113
